# Prereqs and config
---

In [1]:
# Standard library
import sys
import os
from enum import Enum

# Scientific computing
import numpy as np
import powerlaw
from sklearn.metrics import normalized_mutual_info_score as nmiscore

# Network science
import networkx as nx
import community as louvain

# Data management
import pandas as pd
from tabulate import tabulate

# Data viz
from IPython.display import Image
import seaborn as sns
import altair as alt
import matplotlib.pyplot as plt
%matplotlib inline

# Miscellaneous
## Path adjustment
sys.path.append("..")

## Imports
from structs import core
from structs import multiplex
from utils import performance

## Aliases
read_edgelist = core.read_edgelist
Multiplex = multiplex.Multiplex
MultiplexCorpus = multiplex.MultiplexCorpus
auc = performance.get_auc

Note: to be able to use all crisp methods, you need to install some additional packages:  {'karateclub', 'graph_tool', 'wurlitzer', 'infomap', 'leidenalg'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer', 'leidenalg', 'infomap'}


In [2]:
# Config
accepted_layers = {
    "aireu": (1, 2, 3),
    "arxiv": (2, 6, 7),
    "celegans": (1, 2, 3),
    "drosophila": (1, 2, 3, 4),
    "london": (1, 2, 3),
}

# Parametrized analysis over corpus
---

## Gathering corpus

In [3]:
# Create multiplex corpus
corpus = []

## Initializing each multiplex
## * Note: Some layers are statistically unsound (too small) and removed
for filepath in os.listdir("../data/multiplexes"):
    filepath = f"../data/multiplexes/{filepath}"
    ### Read from edgelist
    multiplex_ = read_edgelist(
        filepath=filepath, network_type="MULTIPLEX", output_type="STANDARD"
    )
    name_ = filepath.split("-")[1].split(".")[0]  # Particular for this naming convention!

    ### Restrict layers, removing small layers or those without precendence
    multiplex_ = {
        key: value for key, value in multiplex_.items()
        if key in accepted_layers[name_]
    }


    ### Instantiate as class and append to corpus
    multiplex_ = Multiplex(multiplex_dict=multiplex_)
    multiplex_.name = name_
    corpus.append(multiplex_)

## Initialize MultiplexCorpus object
corpus = MultiplexCorpus(corpus)

## Applying workflow

### Gathering summary tables

In [4]:
# Gather summary dataframes
## Initialize empty dataframes
df_layers = pd.DataFrame({})
df_pairs = pd.DataFrame({})

## Fill in each multiplexes' contribution
for multiplex_ in corpus.corpus:
    ### Gather layer-wise information
    df_ = multiplex_.summarize_multiplex_layers()
    df_["Name"] = [multiplex_.name] * len(df_)
    df_layers = pd.concat([df_layers, df_])

    ### Gather layer pair information
    df_ = multiplex_.summarize_multiplex_pairs()
    df_["Name"] = [multiplex_.name] * len(df_)
    df_pairs = pd.concat([df_pairs, df_])

## Reorder for convenience
df_layers = df_layers[["Name", "Layer", "ActiveNodeCount", "ActiveEdgesCount", "Modularity", "ComponentsCount", "Heterogeneity"]]
df_pairs = df_pairs[["Name", "LayerLeft", "LayerRight", "NodeOverlap", "EdgeOverlap", "AverageDegreeRatio", "DegreeSequenceCorrelation", "NMI"]]

In [6]:
df_layers

,Name,Layer,ActiveNodeCount,ActiveEdgesCount,Modularity,ComponentsCount,Heterogeneity
0,london,1,271,312,0.832270,1,1.180391
1,london,2,83,83,0.776165,1,1.072289
2,london,3,45,46,0.694707,1,1.095227
0,aireu,1,106,244,0.245482,1,6.852896
1,aireu,2,128,601,0.232346,1,2.513747
2,aireu,3,99,307,0.300979,1,3.105009
0,arxiv,1,5465,14485,0.858809,508,2.505357
1,arxiv,2,3506,7341,0.908415,394,2.128181
2,arxiv,3,1451,2582,0.960252,259,1.822912
0,drosophila,1,7356,23977,0.450956,57,3.558654


### Appending reconstruction performances

In [7]:
# Add performance summary to pairs dataframe
# * performance summary here is AUC of reconstruction accuracy as a function of training set size
## Read in performance dataframe
perfs = pd.read_csv("../data/accuracy_networks_concat.csv")
perfs = perfs.rename(columns={"Multiplex": "Name"})

## Prepare the x_coords of performance curve
pfis = sorted(list(set(perfs["PFI"])))

## Calculate AUCs
aucs = []
for row in df_pairs.iterrows():  # Yeah yeah, I know iterrows is not preferred.
    series_ = dict(row[1])
    y_coords = [
        np.mean(
            perfs  # Yeah yeah pt. 2, I know its inefficient queries. It's _legible_ though
            .query("Name == @series_['Name']")
            .query("LayerLeft == @series_['LayerLeft'] + 1")
            .query("LayerRight == @series_['LayerRight'] + 1")
            .query("PFI == @pfi")
            ["Accuracy"]
        )
        for pfi in pfis
    ]
    aucs.append(auc(pfis, y_coords))

## Add resultant AUCs to pairs dataframe
df_pairs["AUC"] = aucs

## Fix an indexing issue
df_pairs["LayerLeft"] = df_pairs["LayerLeft"].apply(lambda x: x + 1)
df_pairs["LayerRight"] = df_pairs["LayerRight"].apply(lambda x: x + 1)

In [8]:
# Rename layers
## * NOTE: Done at this stage to avoid issues with calculations above
def adjust_layer_names(df_line):
    aireu_ = {1: "Lufthansa", 2: "Ryanair", 3: "EasyJet"}
    arxiv_ = {1: "physics.data-an", 2: "cond-mat.dis-nn", 3: "cond-mat.stat-mech"}
    celegans_ = {1: "Electric", 2: "Chemical Monadic", 3: "Chemical Polyadic"}
    drosophila_ = {1: "Direct", 2: "Suppressive", 3: "Additive", 4: "Physical association"}
    london_ = {1: "Underground", 2: "Overground", 3: "DLR"}


    new_ = eval(
        f"{df_line['Name']}_[{df_line['Layer']}]"
    )

    return new_

def adjust_layerpairs_left(df_line):
    aireu_ = {1: "Lufthansa", 2: "Ryanair", 3: "EasyJet"}
    arxiv_ = {1: "physics.data-an", 2: "cond-mat.dis-nn", 3: "cond-mat.stat-mech"}
    celegans_ = {1: "Electric", 2: "Chemical Monadic", 3: "Chemical Polyadic"}
    drosophila_ = {1: "Direct", 2: "Suppressive", 3: "Additive", 4: "Physical association"}
    london_ = {1: "Underground", 2: "Overground", 3: "DLR"}


    new_ = eval(
        f"{df_line['Name']}_[{df_line['LayerLeft']}]"
    )

    return new_

def adjust_layerpairs_right(df_line):
    aireu_ = {1: "Lufthansa", 2: "Ryanair", 3: "EasyJet"}
    arxiv_ = {1: "physics.data-an", 2: "cond-mat.dis-nn", 3: "cond-mat.stat-mech"}
    celegans_ = {1: "Electric", 2: "Chemical Monadic", 3: "Chemical Polyadic"}
    drosophila_ = {1: "Direct", 2: "Suppressive", 3: "Additive", 4: "Physical association"}
    london_ = {1: "Underground", 2: "Overground", 3: "DLR"}


    new_ = eval(
        f"{df_line['Name']}_[{df_line['LayerRight']}]"
    )

    return new_

## Adjust layerwise summary dataframe layer names
df_layers["_Layer"] = df_layers["Layer"].copy()
df_layers["Layer"] = df_layers.apply(adjust_layer_names, axis=1)

## Adjust layerpairs summary dataframe layer names
df_pairs["_LayerLeft"] = df_pairs["LayerLeft"].copy()
df_pairs["LayerLeft"] = df_pairs.apply(adjust_layerpairs_left, axis=1)
df_pairs["_LayerRight"] = df_pairs["LayerRight"].copy()
df_pairs["LayerRight"] = df_pairs.apply(adjust_layerpairs_right, axis=1)

# Saving to file
df_layers.to_csv("structural-analysis_layerwise-measures.csv")
df_pairs.to_csv("structural-analysis_layerpairs-measures.csv")

In [10]:
df_pairs

,Name,LayerLeft,LayerRight,NodeOverlap,EdgeOverlap,AverageDegreeRatio,DegreeSequenceCorrelation,NMI,AUC,_LayerLeft,_LayerRight
0,london,Underground,Overground,0.072727,0.023316,0.266026,0.347786,0.679824,0.692296,1,2
1,london,Underground,DLR,0.016077,0.000000,0.147436,0.319949,0.676250,0.719291,1,3
2,london,Overground,DLR,0.015873,0.000000,0.554217,0.641802,0.712373,0.685324,2,3
0,aireu,Lufthansa,Ryanair,0.181818,0.000000,0.405990,0.147368,0.529783,0.896708,1,2
1,aireu,Lufthansa,EasyJet,0.331169,0.009158,0.794788,0.551841,0.478040,0.868603,1,3
2,aireu,Ryanair,EasyJet,0.233696,0.007769,0.510815,0.371397,0.510165,0.845812,2,3
0,arxiv,physics.data-an,cond-mat.dis-nn,0.184916,0.104555,0.506800,0.380855,0.748261,0.876133,1,2
1,arxiv,physics.data-an,cond-mat.stat-mech,0.114226,0.048277,0.178253,0.351704,0.747131,0.822510,1,3
2,arxiv,cond-mat.dis-nn,cond-mat.stat-mech,0.111934,0.034724,0.351723,0.419961,0.778937,0.847994,2,3
0,drosophila,Direct,Suppressive,0.089906,0.001589,0.077741,0.231122,0.489965,0.885521,1,2


In [7]:
# Exploring structural indicators of performance
## Sourcing data
source = df_pairs
range_ = np.linspace(0.5, 1.0, 10)
name_filter = alt.selection_multi(encodings=["color"])


## Defining charts
average_performance = alt.Chart(source).mark_bar().encode(
    alt.X('Name:N'),
    alt.Y('mean(AUC):Q', scale=alt.Scale(domain=[0.5,1.0])),
    color=alt.Color("Name:N"),
).properties(
    width=400,
    height=400
).add_selection(
    name_filter
)

performance_v_nodeoverlap = alt.Chart(source).mark_point(filled=True).encode(
    alt.X('NodeOverlap:Q'),
    alt.Y('AUC:Q', scale=alt.Scale(domain=[0.5,1.0])),
    color=alt.Color("Name:N"),
    tooltip=["LayerLeft", "LayerRight"]
).properties(
    width=400,
    height=400
)

performance_v_edgeoverlap = alt.Chart(source).mark_point(filled=True).encode(
    alt.X('EdgeOverlap:Q'),
    alt.Y('AUC:Q', scale=alt.Scale(domain=[0.5,1.0])),
    alt.Color("Name:N"),
    tooltip=["LayerLeft", "LayerRight"]
).properties(
    width=400,
    height=400
)

performance_v_ratio = alt.Chart(source).mark_point(filled=True).encode(
    alt.X('AverageDegreeRatio:Q'),
    alt.Y('AUC:Q', scale=alt.Scale(domain=[0.5,1.0])),
    alt.Color("Name:N"),
    tooltip=["LayerLeft", "LayerRight"]
).properties(
    width=400,
    height=400
)

performance_v_nmi = alt.Chart(source).mark_point(filled=True).encode(
    alt.X('NMI:Q'),
    alt.Y('AUC:Q', scale=alt.Scale(domain=[0.5,1.0])),
    alt.Color("Name:N"),
    tooltip=["LayerLeft", "LayerRight"]
).properties(
    width=400,
    height=400
)


## Concatenating and plotting charts
average_performance | alt.vconcat(
    performance_v_nodeoverlap.transform_filter(name_filter) | performance_v_edgeoverlap.transform_filter(name_filter),
    performance_v_ratio.transform_filter(name_filter) | performance_v_nmi.transform_filter(name_filter)
).resolve_axis(
    y="independent"
).resolve_scale(
    y="independent"
)

alt.HConcatChart(...)

In [8]:
alt.vconcat(
    performance_v_nodeoverlap | performance_v_edgeoverlap,
    performance_v_ratio | performance_v_nmi
).resolve_axis(
    y="independent"
).resolve_scale(
    y="independent"
)

alt.VConcatChart(...)

In [9]:
performance_v_edgeoverlap | performance_v_nmi

alt.HConcatChart(...)